<a href="https://colab.research.google.com/github/mahmud-sayed-alamin/GNN-Water-mark/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

In [ ]:
torch.manual_seed(42)

In [17]:
!git checkout -b main

Switched to a new branch 'main'


In [22]:
%cd content


[Errno 2] No such file or directory: 'content'
/root


In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.05, training=self.training)

        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
model = GCN(
    in_channels=1433,
    hidden_channels=32,
    out_channels=7
)

In [ ]:
out = model(data.x, data.edge_index)
print(out.shape)

torch.Size([2708, 7])


In [ ]:
import torch.nn.functional as F

def compute_loss(output, y, mask):
    loss = F.nll_loss(output[mask], y[mask])
    return loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-2)

In [ ]:
def train(data):
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = compute_loss(output, data.y, data.train_mask)
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
def test(data):
    model.eval()
    output = model(data.x, data.edge_index)
    pred = output.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        correct = pred[mask].eq(data.y[mask]).sum().item()
        acc = correct / mask.sum().item()
        accs.append(acc)
    return accs

In [ ]:
for epoch in range(200):
    loss = train(data)
    train_acc, val_acc, test_acc = test(data)

    if epoch % 10 == 0:
        print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, "
              f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")

Epoch: 000, Loss: 1.9519, Train Acc: 0.7214, Val Acc: 0.4000, Test Acc: 0.3870
Epoch: 010, Loss: 1.0011, Train Acc: 0.9643, Val Acc: 0.7700, Test Acc: 0.8040
Epoch: 020, Loss: 0.7521, Train Acc: 0.9786, Val Acc: 0.7880, Test Acc: 0.8190
Epoch: 030, Loss: 0.6811, Train Acc: 0.9857, Val Acc: 0.7860, Test Acc: 0.8110
Epoch: 040, Loss: 0.7163, Train Acc: 0.9857, Val Acc: 0.7860, Test Acc: 0.8170
Epoch: 050, Loss: 0.7496, Train Acc: 0.9786, Val Acc: 0.7840, Test Acc: 0.8080
Epoch: 060, Loss: 0.7324, Train Acc: 0.9714, Val Acc: 0.7860, Test Acc: 0.8100
Epoch: 070, Loss: 0.7272, Train Acc: 0.9786, Val Acc: 0.7820, Test Acc: 0.8140
Epoch: 080, Loss: 0.7329, Train Acc: 0.9786, Val Acc: 0.7840, Test Acc: 0.8140
Epoch: 090, Loss: 0.7226, Train Acc: 0.9857, Val Acc: 0.7840, Test Acc: 0.8140
Epoch: 100, Loss: 0.7146, Train Acc: 0.9857, Val Acc: 0.7780, Test Acc: 0.8100
Epoch: 110, Loss: 0.7312, Train Acc: 0.9857, Val Acc: 0.7800, Test Acc: 0.8070
Epoch: 120, Loss: 0.7267, Train Acc: 0.9857, Val Acc